In [12]:
import cv2  # Import OpenCV for video capturing and image processing
import mediapipe as mp  # Import Mediapipe for hand tracking
import time  # Import time for calculating FPS
import math  # Import math for calculations
import numpy as np  # Import numpy for numerical operations

In [13]:
class HandTrackingDynamic:
    def __init__(self, mode=False, maxHands=1, detectionCon=0.5, trackCon=0.5):
        # Initialize Mediapipe Hands with parameters for detection and tracking
        self.__mode__ = mode  # Static or dynamic mode for hand detection
        self.__maxHands__ = maxHands  # Maximum number of hands to detect
        self.__detectionCon__ = detectionCon  # Minimum detection confidence
        self.__trackCon__ = trackCon  # Minimum tracking confidence
        self.handsMp = mp.solutions.hands  # Mediapipe Hands solution
        self.hands = self.handsMp.Hands(static_image_mode=mode, max_num_hands=maxHands, 
                                        min_detection_confidence=detectionCon, min_tracking_confidence=trackCon)
        self.mpDraw = mp.solutions.drawing_utils  # Utility to draw hand landmarks
        self.tipIds = [4, 8, 12, 16, 20]  # IDs of fingertips for thumb and fingers
        self.startPoint = None  # To store the starting point of a gesture
        self.endPoint = None  # To store the ending point of a gesture
        self.movementCommand = None  # To store the detected movement command
        self.thetaList = []
        self.positiveThetaList = []
        self.negativeThetaList = []
        self.directionX = []
        self.data_matrix = np.zeros((20, 75))
        self.countLists = 0
        self.fiveThetaLists = []
        self.currentMatrixRow = 0

    def findFingers(self, frame, draw=True):
        # Process the frame to detect hands
        imgRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert frame to RGB
        self.results = self.hands.process(imgRGB)  # Process the frame with Mediapipe Hands
        if self.results.multi_hand_landmarks:  # If hands are detected
            for handLms in self.results.multi_hand_landmarks:  # Iterate through detected hands
                if draw:  # If drawing is enabled
                    self.mpDraw.draw_landmarks(frame, handLms, self.handsMp.HAND_CONNECTIONS)  # Draw hand landmarks
        return frame  # Return the processed frame

    def findPosition(self, frame, handNo=0, draw=True):
        # Find the position of hand landmarks
        xList = []  # List to store x-coordinates of landmarks
        yList = []  # List to store y-coordinates of landmarks
        bbox = []  # Bounding box around the hand
        self.lmsList = []  # List to store landmark IDs and their coordinates

        if self.results.multi_hand_landmarks:  # If hands are detected
            myHand = self.results.multi_hand_landmarks[handNo]  # Get the specified hand
            for id, lm in enumerate(myHand.landmark):  # Iterate through landmarks
                h, w, c = frame.shape  # Get frame dimensions
                cx, cy = int(lm.x * w), int(lm.y * h)  # Convert normalized coordinates to pixel values
                xList.append(cx)  # Append x-coordinate to the list
                yList.append(cy)  # Append y-coordinate to the list
                self.lmsList.append([id, cx, cy])  # Store the landmark ID and coordinates
                # print(self.lmsList[0])
                # print(self.lmsList[20])

                if draw:  # If drawing is enabled
                    cv2.circle(frame, (cx, cy), 5, (255, 0, 255), cv2.FILLED)  # Draw circles at landmarks

            xmin, xmax = min(xList), max(xList)  # Find min and max x-coordinates
            ymin, ymax = min(yList), max(yList)  # Find min and max y-coordinates
            bbox = xmin, ymin, xmax, ymax  # Define bounding box

            if draw:  # If drawing is enabled
                cv2.rectangle(frame, (xmin - 20, ymin - 20), (xmax + 20, ymax + 20),
                              (0, 255, 0), 2)  # Draw bounding box

        return self.lmsList, bbox  # Return landmark list and bounding box

    def findFingerUp(self):
        # Determine which fingers are up
        fingers = []  # List to store finger states
        if self.lmsList:  # If landmarks are available
            handType = "Right Hand" if self.lmsList[0][1] > self.lmsList[1][1] else "Left Hand"
            # Check thumb (left or right based on position)
            if self.lmsList[self.tipIds[0]][1] > self.lmsList[self.tipIds[0] - 1][1]:
                fingers.append(1)  # Thumb is up
            else:
                fingers.append(0)  # Thumb is down

            # Check other fingers (vertical position)
            for id in range(1, 5):
                if self.lmsList[self.tipIds[id]][2] < self.lmsList[self.tipIds[id] - 2][2]:
                    fingers.append(1)  # Finger is up
                else:
                    fingers.append(0)  # Finger is down
        return fingers  # Return list of finger states

    def detectCommand(self):
        # Detect the command based on start and end points
        if not self.startPoint or not self.endPoint:  # If either point is missing
            return None  # No command

        dx = self.endPoint[0] - self.startPoint[0]  # Calculate x-axis movement
        dy = self.endPoint[1] - self.startPoint[1]  # Calculate y-axis movement

        # INFLECTION_THRESHOLD = 250

        palmCoordinateX = self.lmsList[0][1]  # Extract X coordinate
        palmCoordinateY = self.lmsList[0][2]  # Extract Y coordinate
        
        # Initialize storage lists if they don't exist
        if not hasattr(self, 'palmListX'):
            self.palmListX = []
            self.palmListY = []
            self.thetaList = []  # Stores past angles
        
        # Append new coordinates
        self.palmListX.append(palmCoordinateX)
        self.palmListY.append(palmCoordinateY)
        
        # Ensure we have at least two points before calculating theta
        if len(self.palmListX) > 1:
            x_2, x_1 = self.palmListX[-1], self.palmListX[-2]  # Last two X values
            y_2, y_1 = self.palmListY[-1], self.palmListY[-2]  # Last two Y values
            if len(self.directionX) < 15:
                self.directionX.append(self.palmListX[-1])
            else:
                # if self.directionX[-1] < self.directionX[0]:
                    # print("LEFT TO RIGHT movement")
                # else:
                    # print("RIGHT TO LEFT movement")
                    
                self.directionX = []
            # Compute the angle (in radians)
            theta1 = math.atan2(y_2 - y_1, x_2 - x_1)
            theta_degrees = math.degrees(theta1)  # Convert to degrees for easier analysis
            # print("THETA", theta_degrees)
     
            if len(self.thetaList) < 15:
                self.thetaList.append(theta_degrees)
            elif len(self.thetaList) == 15:
                
                positive_count = len([num for num in self.thetaList if num > 0])
                negative_count = len([num for num in self.thetaList if num < 0])

                # print("Positive numbers:", positive_count)
                # print("Negative numbers:", negative_count)
                if positive_count > negative_count:
                    for num in self.thetaList:
                        if num > 0:
                            self.positiveThetaList.append(num)
                        
                    # print("Most numbers are positive. The direction is from top to bottom.")
                    averagePositive = np.mean(self.positiveThetaList)
                    # print("Average Positive:", averagePositive)
                    
                elif negative_count > positive_count:
                    for num in self.thetaList:
                        if num < 0:
                            self.negativeThetaList.append(num)
                
                    # print("Most numbers are negative. The direction is from bottom to top.")
    
                    averageNegative = np.mean(self.negativeThetaList)
                    
                    # print("Average Negative:", averageNegative)
                if self.countLists < 5:
                    self.fiveThetaLists.append(self.thetaList.copy())
                    self.countLists += 1
                elif self.countLists == 5:
                    print("PROVERQVAM ", self.fiveThetaLists)
                    if self.currentMatrixRow <= 19:
                        self.saveMatrixRow(self.currentMatrixRow)
                        print("ROW ", self.currentMatrixRow, " of data_matrix saved.")
                        self.currentMatrixRow += 1
                        self.countLists = 0
                        # while self.movementCommand != "STOP":
                        #     print("Waiting for STOP command...")
                        #     pass
                self.thetaList = []
                
            # self.thetaList.append(theta_degrees)  # Store angle in list
        
            # # Detect Inflection Point (Need at least 3 angles to check curvature change)
            # if len(self.thetaList) > 2:
            #     prev_theta_2 = self.thetaList[-3]  # Two steps ago
            #     prev_theta_1 = self.thetaList[-2]  # One step ago
            #     curr_theta = self.thetaList[-1]  # Current angle
            # 
            #     diff1 = abs(prev_theta_1 - prev_theta_2)
            #     diff2 = abs(curr_theta - prev_theta_1)
            #     
            #     # Inflection occurs if the trend changes direction & the change is above the threshold
            #     if ((prev_theta_2 < prev_theta_1 > curr_theta) or (prev_theta_2 > prev_theta_1 < curr_theta)) and \
            #         (diff1 > INFLECTION_THRESHOLD and diff2 > INFLECTION_THRESHOLD):
            #         print(f"Inflection Point Detected = {curr_theta:.2f}° (Δ1 = {diff1:.2f}°, Δ2 = {diff2:.2f}°)")
                
        thumbCoordinateX = self.lmsList[4]
        pinkyCoordinateX = self.lmsList[20]
        thumbPinkyDistanceLeftHand = (thumbCoordinateX[1] - pinkyCoordinateX[1])
        thumbPinkyDistanceRightHand = (pinkyCoordinateX[1] - thumbCoordinateX[1])
        
        ringFingerCoordinateY = self.lmsList[16]
        middleFingerCoordinateY = self.lmsList[12]
        ringMiddleDistance = (ringFingerCoordinateY[2] - middleFingerCoordinateY[2])
        # ringMiddleDistanceLeftHand = (ringFingerCoordinateY[2] - middleFingerCoordinateY[2])
        # ringMiddleDistanceRightHand = (middleFingerCoordinateY[2] - ringMiddleDistanceLeftHand[2])
        
        # print(self.handType)
        
        if abs(dx) > abs(dy):  # Horizontal movement
            if dy > 1 and 0 <= thumbPinkyDistanceLeftHand < 50 and ringMiddleDistance < 20 and self.handType == "Left Hand":
                    return "Turn Left"
            elif dy > 1 and 0 <= thumbPinkyDistanceRightHand < 50 and ringMiddleDistance < 20 and self.handType == "Right Hand":
                    return "Turn Right"
            elif dx > 100:  # Threshold for significant movement
                valX = dx/210
                return "MOVE LEFT"  # Command: Move Left
            if dy < -1 and 0 <= thumbPinkyDistanceLeftHand < 50 and self.handType == "Left Hand":
                return "Turn Left"
            elif dy < -1 and 0 <= thumbPinkyDistanceRightHand < 50 and self.handType == "Right Hand":
                return "Turn Right"
            elif dx < -100:
                valX = dx/210
                return "MOVE RIGHT"  # Command: Move Right
        else:  # Vertical movement
            if dy > 1 and 0 <= thumbPinkyDistanceLeftHand < 50 and ringMiddleDistance < 20 and  self.handType == "Left Hand":
                    return "Turn Left"
            elif dy > 1 and 0 <= thumbPinkyDistanceRightHand < 50 and ringMiddleDistance < 20 and self.handType == "Right Hand":
                    return "Turn Right"
            elif dy > 100:
                valY = dy/210
                return "MOVE BACKWARDS"  # Command: Move Backwards
            if dy < -1 and 0 <= thumbPinkyDistanceLeftHand < 50 and ringMiddleDistance < 20 and self.handType == "Left Hand":
                    return "Turn Left"
            elif dy < -1 and 0 <= thumbPinkyDistanceRightHand < 50 and ringMiddleDistance < 20 and self.handType == "Right Hand":
                    return "Turn Right"
            elif dy < -100:
                valY = dy/210
                return "MOVE STRAIGHT" # Command: Move Straight
            

        return None  # No significant movement

    def processMovement(self):
        # Process the movement and detect commands
        fingers = self.findFingerUp()  # Get finger states
        if self.lmsList:
            self.handType = "Right Hand" if self.lmsList[0][1] > self.lmsList[1][1] else "Left Hand"
            
        if sum(fingers) == 0:  # All fingers down (fist)
            self.movementCommand = "STOP"  # Command: Stop
            self.startPoint = None  # Reset start point
            self.endPoint = None  # Reset end point
        elif self.lmsList:  # If landmarks are detected
            palmX, palmY = self.lmsList[0][1:]  # Palm center (landmark 0)

            if not self.startPoint:  # If no start point
                self.startPoint = (palmX, palmY)  # Set start point
            else:  # If start point exists
                self.endPoint = (palmX, palmY)  # Update end point
                self.movementCommand = self.detectCommand()  # Detect command

        
    def saveMatrixRow(self, rowIndex):
    # Store the current 15 theta values
        self.fiveThetaLists.append(self.thetaList.copy())

    # Only proceed if we have 5 lists (i.e., 75 values)
        if len(self.fiveThetaLists) == 5:
        # Flatten the list of lists
            flattened = [theta for sublist in self.fiveThetaLists for theta in sublist]

            if len(flattened) == 75:
                self.data_matrix[rowIndex] = flattened
                print(f"Row {rowIndex} of data_matrix saved.")
            else:
                print(f"ERROR: Flattened data is not 75 elements long.")

        # Reset for next row
            self.fiveThetaLists = []


        
        # return self.movementCommand, self.handType  # Return the movement command

In [14]:
def main():
    ctime = 0  # Current time for FPS calculation
    ptime = 0  # Previous time for FPS calculation
    cap = cv2.VideoCapture(0)  # Open the webcam
    detector = HandTrackingDynamic()  # Initialize the hand tracking class
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)  # Set video width
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)  # Set video height

    if not cap.isOpened():  # Check if the camera opened successfully
        print("Cannot open camera")  # Print error message
        exit()  # Exit the program

    while True:  # Loop to process video frames
        ret, frame = cap.read()  # Read a frame from the webcam
        if not ret:  # If frame is not captured
            print("Failed to grab frame")  # Print error message
            break  # Exit the loop

        frame = detector.findFingers(frame)  # Detect and draw hand landmarks
        lmsList, _ = detector.findPosition(frame)  # Get landmark positions
        command = detector.processMovement()  # Process movement and get command
        # print(detector.movementCommand)
        # if hasattr(detector, "handType"):
        #     if detector.handType == "Left Hand":
        #         print("Right Hand")
        #     elif detector.handType == "Right Hand":
        #         print("Left Hand")
            
        if detector.movementCommand:  # If a command is detected
            cv2.putText(frame, detector.movementCommand, (10, 70), cv2.FONT_HERSHEY_PLAIN, 3, (0, 0, 255), 3)  # Display the command

        ctime = time.time()  # Get current time
        fps = 1 / (ctime - ptime) if (ctime - ptime) > 0 else 0  # Calculate FPS
        ptime = ctime  # Update previous time

        cv2.putText(frame, f"FPS: {int(fps)}", (10, 120), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 2)  # Display FPS

        cv2.imshow('Hand Gesture Control', frame)  # Show the video frame
        if cv2.waitKey(1) & 0xFF == ord('q'):  # Break loop on 'q' key press
            break

    cap.release()  # Release the webcam
    cv2.destroyAllWindows()  # Close all OpenCV windows
    

In [15]:
if __name__ == "__main__":
    main()  # Run the main function

PROVERQVAM  [[-135.0, -63.43494882292201, -113.49856567595211, -71.56505117707799, -83.6598082540901, -104.03624346792648, -110.55604521958347, -108.43494882292202, -71.56505117707799, 26.56505117707799, 0.0, 45.0, -45.0, 26.56505117707799, 0.0], [-90.0, 0.0, 0.0, -68.19859051364818, -71.56505117707799, -68.19859051364818, -53.13010235415598, -75.96375653207353, -78.69006752597979, -116.56505117707799, -71.56505117707799, -63.43494882292201, -135.0, -135.0, -90.0], [-75.96375653207353, -104.03624346792648, -90.0, -90.0, -97.1250163489018, -80.53767779197439, -99.46232220802563, -82.8749836510982, -99.46232220802563, -68.19859051364818, -80.53767779197439, -90.0, -63.43494882292201, -104.03624346792648, -95.71059313749964], [-90.0, -84.8055710922652, -101.30993247402021, -81.86989764584403, -77.47119229084849, -90.0, -71.56505117707799, -96.3401917459099, -97.1250163489018, -81.86989764584403, -113.96248897457819, -119.74488129694222, -108.43494882292202, -139.39870535499554, -105.94539